In [8]:
import pandas as pd

df = pd.read_csv("04_final_output.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"], format="ISO8601")
threshold = pd.Timedelta(milliseconds=4)

last_occ = {}
dup_analysis = {} # i -> SYM, OCC, TS, Price_low, Price_high, TimeDiff

for i, (sym, time, price) in enumerate(zip(df["symbol"], df["timestamp"], df["price"])):
    if sym not in last_occ:
        pass
    else:
        if abs(time - df["timestamp"][last_occ[sym]]) <= threshold:
            if last_occ[sym] in dup_analysis:
                dup_analysis[last_occ[sym]][1] += 1
                if (time != dup_analysis[last_occ[sym]][2]):
                    dup_analysis[last_occ[sym]][5] = False

                if price > dup_analysis[last_occ[sym]][4]:
                    dup_analysis[last_occ[sym]][4] = price

                if price < dup_analysis[last_occ[sym]][3]:
                    dup_analysis[last_occ[sym]][3] = price

            else:
                dup_analysis[i] = [sym, 2, df["timestamp"][last_occ[sym]], min(price, df["price"][last_occ[sym]]), max(price, df["price"][last_occ[sym]]), time == df["timestamp"][last_occ[sym]]]

    last_occ[sym] = i


lst=[]
for key in dup_analysis:
    lst.append(dup_analysis[key])

analysis_df = pd.DataFrame(lst, columns=["Symbol", "Occurrences", "Timestamp", "Price_low", "Price_high", "TimeDiff"])
analysis_df.to_csv("duplication_analysis.csv", index=False)
